In [ ]:
code = 'SRE_PSL'
pickle_path = 'C:/PICKLE/'
parameter_path = f'Parameter_{code}.csv'
meta_data_path = f"Parameter_{code}_MetaData.csv"
output_csv_path = f'{code}_output\\'

from pgcbacktest.BtParameters import *
from pgcbacktest.BacktestOptions import *

try:
    parameter, parameter_len = get_parameter_data(code, parameter_path)
    meta_data, meta_row_nos = get_meta_data(code, meta_data_path)
    os.makedirs(output_csv_path, exist_ok=True)
except Exception as e:
    input(str(e))

In [ ]:
def find_vwap(t_df):
    df = pd.DataFrame()
    
    df['date_time'] = t_df['date_time']
    df['CE_price'] = t_df['CE_price']
    df['PE_price'] = t_df['PE_price']
    df['straddle_price'] = df['CE_price'] + df['PE_price']
    
    df['CE_volume'] = t_df['CE_volume']
    df['PE_volume'] = t_df['PE_volume']
    
    df['ce_pv'] = t_df['CE_price'] * t_df['CE_volume']  
    df['pe_pv'] = t_df['PE_price'] * t_df['PE_volume']
    
    df['total_pv'] = df['ce_pv'] + df['pe_pv']
    df['total_vol'] = df['CE_volume'] + df['PE_volume']
    
    # Cumulative sums
    df['cum_pv'] = df['total_pv'].cumsum()
    df['cum_vol'] = df['total_vol'].cumsum()
    
    #     # Cumulative straddle VWAP
    df['cumulative_straddle_vwap'] = df['cum_pv'] / df['cum_vol']
    
    return df

In [ ]:
def SRE_per_minute_mtm(bt, start_time, end_time, orderside, sl, intra_sl, om, vwap, re_entries, seperate=False):
    try:
        start_dt = datetime.datetime.combine(bt.current_date, start_time)
        end_dt = datetime.datetime.combine(bt.current_date, end_time)

        ce_scrip, pe_scrip, ce_price, pe_price, future_price, start_dt = bt.get_strike(start_dt, end_dt, om=om)
        if ce_scrip is None: return None

        entry_time = start_dt
        std_sl_time, std_mtm_data = bt.sl_check_combine_leg(start_dt, end_dt, ce_scrip, pe_scrip, sl=sl, intra_sl=intra_sl, orderside=orderside, per_minute_mtm=True)
        
        std_mtm_data0 = set_pm_time_index(std_mtm_data, time_index)
        re_std_mtm_data = set_pm_time_index(pd.Series(), time_index)
        
        for re_no in range(max_re):
            
            if std_sl_time and re_no < re_entries and (std_sl_time < end_dt - datetime.timedelta(minutes=5)):
                start_dt = std_sl_time
                ce_scrip, pe_scrip, ce_price, pe_price, _, start_dt = bt.get_strike(start_dt, end_dt, om=om)
                
                if ce_scrip is None:
                    std_sl_time = ''
                    continue

                std_sl_time, std_mtm_data = bt.sl_check_combine_leg(start_dt, end_dt, ce_scrip, pe_scrip, sl=sl, intra_sl=intra_sl, orderside=orderside, per_minute_mtm=True)
                std_mtm_data = set_pm_time_index(std_mtm_data, time_index)
                re_std_mtm_data += std_mtm_data
            else:
                break
        
        if seperate:
            return std_mtm_data0, re_std_mtm_data
        else:
            return std_mtm_data0 + re_std_mtm_data

    except Exception as e:
        print(e, [bt.index, bt.current_date, start_time, end_time, orderside, sl, intra_sl, om, vwap, re_entries])
        return

In [ ]:
def SRE_PSL(bt, start_time, end_time, last_trade_time, trade_interval, orderside, sl, intra_sl, om, vwap, re_entries):
    try:
        start_dt = datetime.datetime.combine(bt.current_date, start_time)
        end_dt = datetime.datetime.combine(bt.current_date, end_time)
        last_trade_dt = datetime.datetime.combine(bt.current_date, last_trade_time)
        
        t_df=pd.DataFrame(columns=['index','date_time', 'ce_scrip','CE_price','CE_volume', 'pe_scrip','PE_price','PE_volume','straddle_price'])
        while start_dt < end_dt-datetime.timedelta(minutes=10) :
            ce_scrip, pe_scrip, ce_price, pe_price, future_price, start_dt = bt.get_strike(start_dt, end_dt, om=0)
            if ce_scrip is None: return None
            tt_options_data = bt.options
            ce_options_data = tt_options_data[(tt_options_data.scrip == ce_scrip)&(tt_options_data.date_time == start_dt)].reset_index(drop=True)
            pe_options_data = tt_options_data[(tt_options_data.scrip == pe_scrip)&(tt_options_data.date_time == start_dt)].reset_index(drop=True)
            ce_volume = ce_options_data.volume.iloc[0]
            pe_volume = pe_options_data.volume.iloc[0]
            straddle_price = ce_price + pe_price
            t_df.loc[len(t_df.index)] = index, start_dt, ce_scrip, ce_price, ce_volume, pe_scrip,pe_price, pe_volume, f'{round(straddle_price,2)}'
            start_dt += datetime.timedelta(minutes=1)
        df = find_vwap(t_df)
        cumulative_straddle_vwap_list, straddle_price_list = list(df.cumulative_straddle_vwap), list(df.straddle_price)
        for idx, (vwap_price, s_price) in enumerate(zip(cumulative_straddle_vwap_list, straddle_price_list)):            
            if s_price <= vwap_price*(1+(vwap/100)):
                t_on = idx
                true_time = (df.loc[idx].date_time)
                start_dt = true_time
                break
        
        entry_time = start_dt
        last_trade_dt = start_dt
        time_range = pd.date_range(start_dt, last_trade_dt, freq=trade_interval.lower()).time
        
        per_minute_trades = [SRE_per_minute_mtm(bt, re_time, end_time, orderside, sl, intra_sl, om, vwap, re_entries) for re_time in time_range]
        per_minute_trades = [t for t in per_minute_trades if t is not None]
        per_minute_mtm = np.sum(per_minute_trades, axis=0)
        
        if only_mtm:
            mtm_time_list = list(per_minute_mtm)
            mtm_time_list += [mtm_time_list[-1]]
        else:
            total_minutes = len(time_range)
            per_minute_margin = int(fund/total_minutes)
            shares_per_minute = int(per_minute_margin/margin_per_share)
            per_minute_mtm_total = per_minute_mtm * shares_per_minute

            mtm_dict ={}
            for mtm_percent, check_mtm in check_mtms.items():

                if check_mtm > 0:
                    condition = np.where(per_minute_mtm_total > check_mtm)[0]
                else:
                    condition = np.where(per_minute_mtm_total < check_mtm)[0]

                try:
                    idx = condition[0]
                    time = time_index[idx]
                    mtm = per_minute_mtm_total[idx+1]
                except:
                    time, mtm = '', per_minute_mtm_total[-1]

                mtm_dict[mtm_percent] = (time, mtm)

            mtm_time_list = [fund] + [item for value in mtm_dict.values() for item in value]
        
        return [code, bt.index, start_time, end_time, last_trade_time, trade_interval, orderside, sl, intra_sl, om, vwap, re_entries, bt.current_date.date(), bt.current_date.day_name(), bt.dte, entry_time.time(), future_price] + mtm_time_list
    except Exception as e:
        print(e, [bt.index, bt.current_date, start_time, end_time, last_trade_time, trade_interval, orderside, sl, intra_sl, om, vwap, re_entries])
        return

In [ ]:
for row_idx in range(len(meta_data)):
    
    if row_idx in meta_row_nos and meta_data.loc[row_idx, 'run']:
        try:
            meta_row = meta_data.iloc[row_idx]
            index, dte, from_date, to_date, start_time, end_time, date_lists = get_meta_row_data(meta_row, pickle_path)
            max_re = 7
            
            log_cols = ('P_Strategy/P_Index/P_StartTime/P_EndTime/P_LastTradeTime/P_TradeInterval/P_OrderSide/P_SL/P_intraSL/P_OM/P_VWAP/P_ReEntries/Date/Day/DTE/EntryTime/Future/')
            
            only_mtm = False
            if only_mtm:
                log_time_col = get_pm_time_index(datetime.datetime.now()).time
                log_cols += '/'.join(map(str, log_time_col))
                log_cols += '/Final.PNL'
            else:
                notinal_value = 12
                fund = 100_00_00_00
                mtm_percent_stop = [-0.1, -0.2, -0.3, -0.4, -0.5, -0.6, -0.7, -0.8, -0.9, -1, -1.25, -1.5, -1.75, -2, -2.5, -3, -3.5, -4, -100, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.25, 1.5, 1.75, 2, 2.5, 3, 3.5, 4]
                log_cols += 'Fund'
                for mtmp in mtm_percent_stop:
                    log_cols += f'/{mtmp:.2f}.Time/{mtmp:.2f}.PNL'

            log_cols = log_cols.split('/')

            for current_date in date_lists:

                file_name = f"{index} {current_date.date()} {code}"
                if not is_file_exists(output_csv_path, file_name, parameter_len):

                    t1 = datetime.datetime.now()
                    print(f"Row-{row_idx} | File-{file_name} | Total-{parameter_len}")
                    
                    bt = IntradayBacktest(pickle_path, index, current_date, dte, start_time, end_time)
                    time_index = get_pm_time_index(bt.current_date)
                    future_price = bt.future_data['close'].iloc[0]
                    
                    if not only_mtm:
                        margin_per_share = future_price * (notinal_value / 100)
                        check_mtms = {mtm_percent: fund * mtm_percent / 100 for mtm_percent in mtm_percent_stop}
                    
                    for idx, i in enumerate(range(0, parameter_len, chunk_size), start=1):
                        chunck_file_name = f"{output_csv_path}{file_name} No-{idx}.parquet"
                        print(chunck_file_name)
                        
                        chunk_parameter = parameter.iloc[i:i+chunk_size]
                        chunk = [SRE_PSL(bt, row.entry_time, row.exit_time, row.last_trade_time, row.trade_interval, row.orderside, row.sl, row.intra_sl, row.om, row.vwap, row.re_entries) for row in tqdm(chunk_parameter.itertuples(), total=len(chunk_parameter), colour='RED')]
                        save_chunk_data(chunk, log_cols, chunck_file_name)
                        
                        del chunk
                        del chunk_parameter
                        gc.collect()

                    del bt
                    gc.collect()
                    
                    t2 = datetime.datetime.now()
                    print(t2-t1)
        except Exception as e:
            input(str(e))